In [1]:
import torch 
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

## Loading in the chess set

In [3]:
chess_data = pd.read_csv( 'modified_games.csv' )

In [4]:
chess_data.drop( ['Unnamed: 0'], axis = 'columns', inplace = True )

In [5]:
chess_target = chess_data[ 'higher_rating_won' ]

In [6]:
chess_target = chess_target.to_numpy()

In [7]:
chess_data.drop( ['higher_rating_won'], axis = 'columns', inplace = True )

In [8]:
chess_data = chess_data.to_numpy()

In [24]:
train_X, test_X, train_Y, test_Y = train_test_split( chess_data, chess_target, test_size = 0.25 )

## Convert into tensors

In [25]:
train_X = torch.from_numpy( train_X ).float()
train_Y = torch.from_numpy( train_Y ).long()

In [26]:
test_X = torch.from_numpy( test_X ).float()
test_Y = torch.from_numpy( test_Y ).long()

In [27]:
train = TensorDataset( train_X, train_Y )

In [28]:
train_loader = DataLoader( train, batch_size = 15, shuffle = True )

## Model Declaration

In [29]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear( 4, 128 )
        self.fc2 = nn.Linear( 128, 2 )
    
    def forward(self, x):
        x = F.relu( self.fc1( x ) )
        x = self.fc2( x )
        return F.log_softmax( x, dim = 0 )

In [30]:
model = Net()

## Training

In [31]:
criterion = nn.CrossEntropyLoss()

In [32]:
optimizer = optim.SGD( model.parameters(), lr = 0.001 )

In [33]:
for epoch in range( 500 ):
    total_loss = 0
    
    for train_x, train_y in train_loader:
        train_x, train_y = Variable( train_x ), Variable( train_y )
        
        optimizer.zero_grad()
        output = model( train_x )
        loss = criterion( output, train_y )
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data.item()
        
    if ( epoch + 1 ) % 60 == 0:
        print( epoch + 1, loss )

60 tensor(0.0951, grad_fn=<NllLossBackward>)
120 tensor(0.0486, grad_fn=<NllLossBackward>)
180 tensor(0.0007, grad_fn=<NllLossBackward>)
240 tensor(0.0060, grad_fn=<NllLossBackward>)
300 tensor(0.0015, grad_fn=<NllLossBackward>)
360 tensor(0.0006, grad_fn=<NllLossBackward>)
420 tensor(0.1724, grad_fn=<NllLossBackward>)
480 tensor(0.0002, grad_fn=<NllLossBackward>)


In [19]:
loss.data.item()

0.006390936207026243

## Check result

In [20]:
test_x, test_y = Variable( test_X ), Variable( test_Y )

In [21]:
result = torch.max( model( test_x ).data , 1 )[1]

In [22]:
accuracy = sum( result.numpy() == test_y.data.numpy() ) / len( test_y.data.numpy() )

In [23]:
accuracy

1.0